In [1]:
#Import Libraries
import pandas as pd
from pathlib import Path

In [2]:
#Import CSV with all data
data_path = Path("historical_data.csv")
historical_data = pd.read_csv(data_path)
historical_data.rename( columns = {"Unnamed: 0":"date"}, inplace = True)
historical_data.head()

,date,XLY,XLP,XLE,XLF,XLV,XLI,XLB,XLK,XLU,10 YR
0,2008-01-02,32.13,28.34,79.75,28.41,34.94,38.37,41.33,26.10,41.98,3.91
1,2008-01-03,31.78,28.22,80.21,28.20,35.18,38.58,41.80,26.08,42.08,3.91
2,2008-01-04,30.78,28.04,78.01,27.39,34.83,37.60,40.74,25.07,42.30,3.88
3,2008-01-07,30.93,28.55,77.04,27.55,35.57,37.34,40.25,24.97,43.25,3.86
4,2008-01-08,30.32,28.38,76.08,26.54,35.81,36.39,39.60,24.21,43.33,3.86


In [3]:
# Define a function that returns the indexes where the 10 yr treasury momentum crossed a certain threshold
def spike_selector(df):
    signal1 = 0
    daycount1 = 0
    indexes = []
    df["mmt10"] = (df["10 YR"] - df["10 YR"].shift(10))
    df["roll_10"] = df["mmt10"].rolling(20).mean()
    no_idx = df.reset_index()
    for index, row in zip(no_idx.index, no_idx["roll_10"]):
        if signal1 == 1:
            daycount1 += 1
            if daycount1 > 90:
                signal1 = 0
                daycount1 = 0
        if row >= 0.187 and signal1 == 0:
            #print (signal1, daycount1)
            indexes.append(index)
            signal1 = 1
    df.drop(columns = ["mmt10", "roll_10"], inplace = True)
    return indexes

In [4]:
#Create empty dictionary to store new dataframes.  Then slice 90 days following each treaury yeild increase selected by the above function
yield_spikes_dict = {}
for index in spike_selector(historical_data):
    yield_spikes_dict[index] = pd.DataFrame
for index in spike_selector(historical_data):
    start = index
    end = start + 90
    yield_spikes_dict[index] = historical_data.iloc[start: end]

In [5]:
print(spike_selector(historical_data))

[269, 360, 498, 736, 1357, 2221, 3292]


In [ ]:
#For loop to update keys - DOES NOT WORK
count = 1
for spike in yield_spikes_dict:
    yield_spikes_dict["spike_" + str(count)] = yield_spikes_dict.pop(spike)
    count +=1

In [6]:
#Updating keys manually
yield_spikes_dict["spike_1"] = yield_spikes_dict.pop(269)
yield_spikes_dict["spike_2"] = yield_spikes_dict.pop(360)
yield_spikes_dict["spike_3"] = yield_spikes_dict.pop(498)
yield_spikes_dict["spike_4"] = yield_spikes_dict.pop(736)
yield_spikes_dict["spike_5"] = yield_spikes_dict.pop(1357)
yield_spikes_dict["spike_6"] = yield_spikes_dict.pop(2221)
yield_spikes_dict["spike_7"] = yield_spikes_dict.pop(3292)

In [7]:
yield_spikes_dict["spike_3"]

,date,XLY,XLP,XLE,XLF,XLV,XLI,XLB,XLK,XLU,10 YR
498,2009-12-29,30.2100,26.7800,57.5000,14.43,31.560,23.53,33.41,23.0400,31.4900,3.82
499,2009-12-30,30.0900,26.7500,57.4300,14.44,31.510,23.42,33.47,23.1300,31.5000,3.80
500,2009-12-31,29.8100,26.4400,56.9700,14.39,31.140,23.13,33.04,22.8900,31.0300,3.85
501,2010-01-04,30.0000,26.7200,58.7800,14.69,31.600,23.57,33.98,23.2400,31.1000,3.85
502,2010-01-05,30.1200,26.6900,59.3600,14.94,31.410,23.65,34.14,23.2400,30.7400,3.77
503,2010-01-06,30.1300,26.6600,60.0200,14.99,31.570,23.70,34.64,22.9900,30.9000,3.85
504,2010-01-07,30.3900,26.6600,59.9000,15.31,31.700,23.96,34.49,22.8900,30.7500,3.85
505,2010-01-08,30.3800,26.5500,60.3100,15.22,31.780,24.34,34.87,23.0400,30.7100,3.83
506,2010-01-11,30.3400,26.6491,60.2000,15.23,31.920,24.61,34.78,22.9600,31.0600,3.85
507,2010-01-12,29.9600,26.7700,59.3000,15.00,31.730,24.41,34.11,22.7000,30.9200,3.74
